# FFmpeg demo: video encoding

In [31]:
# If your ./model/vmaf_v0.6.1.json file exists, DONT RUN THIS!

# Converts pickle to json
"""
Pickle2JSON is a simple Python Command Line program for converting Pickle file to JSON file.
Arguments: Only one (1) argument is expected which is the pickle file.
Usage: python pickle2json.py myfile.pkl
Output: The output is a JSON file bearing the same filename containing the JSON document of the converted Pickle file.
"""

# import libraries
import pickle
import json
import sys
import os

# open pickle file
with open('./model/vmaf_v0.6.1.pkl', 'rb') as infile:
    obj = pickle.load(infile)

# convert pickle object to json object
json_obj = json.loads(json.dumps(obj, default=str))

# write the json file
with open(
        './model/vmaf_v0.6.1.json',
        'w',
        encoding='utf-8'
    ) as outfile:
    json.dump(json_obj, outfile, ensure_ascii=False, indent=4)

!ls -al ./model/vmaf*.json
!chmod u+rwx ./model/vmaf*.json

-rwxrw-r-- 1 naveen naveen 1784 May  8 11:43 ./model/vmaf_v0.6.1.json


In [4]:
# Download a static FFmpeg build and add it to PATH.
%run 'util/load-ffmpeg.ipynb'
print('Done!')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 38.9M  100 38.9M    0     0  1869k      0  0:00:21  0:00:21 --:--:-- 2826k
sh: 0: Can't open ffmpeg-*-static/configure
env: PATH=/home/naveen/installations/anaconda3/condabin:/home/naveen/platform-tools:/home/naveen/.cargo/bin:/home/naveen/.local/bin:/home/naveen/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/usr/lib/jvm/java-15-oracle/bin:/usr/lib/jvm/java-15-oracle/db/bin:/home/naveen/.local/bin:/home/naveen/bin:./util/ffmpeg-5.0.1-amd64-static
./util/ffmpeg-5.0.1-amd64-static/ffmpeg
Done!


In [8]:
# [optional] Print input video information. You can upload another video file and change the file name below.
!ffprobe -hide_banner input.mp4
print('Done!')

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'input.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
  Duration: 00:00:10.02, start: 0.000000, bitrate: 2018 kb/s
  Stream #0:0[0x1](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(progressive), 864x360 [SAR 1:1 DAR 12:5], 1885 kb/s, 24 fps, 24 tbr, 12288 tbn (default)
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
  Stream #0:1[0x2](und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 128 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
      vendor_id       : [0][0][0][0]
Done!


In [9]:
# Compress input.mp4 to a target bitrate and save as output.mp4. 
# Change the command line flags as you need for different encoding settings.

!ffmpeg -hide_banner -i input.mp4 -b:v 300k -y output.mp4
print('Done!')

Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'input.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf58.29.100
  Duration: 00:00:10.02, start: 0.000000, bitrate: 2018 kb/s
  Stream #0:0[0x1](und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(progressive), 864x360 [SAR 1:1 DAR 12:5], 1885 kb/s, 24 fps, 24 tbr, 12288 tbn (default)
    Metadata:
      handler_name    : VideoHandler
      vendor_id       : [0][0][0][0]
  Stream #0:1[0x2](und): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, stereo, fltp, 128 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
      vendor_id       : [0][0][0][0]
Stream mapping:
  Stream #0:0 -> #0:0 (h264 (native) -> h264 (libx264))
  Stream #0:1 -> #0:1 (aac (native) -> aac (native))
Press [q] to stop, [?] for help
[libx264 @ 0x5dca440] using SAR=1/1
[libx264 @ 0x5dca440] using cpu capabilities: MMX2 SSE2Fast SSSE3 SSE4.2 AVX FMA3 BMI2 AVX2
[libx264 @ 0x5dca

In [10]:
# Display both the source (top) video and encoded (bottom) video.
# Sometimes the two players get out of sync. Press "sync" to mitigate that.

%run util/play-video.ipynb --ref input.mp4 --tar output.mp4
HTML(html)

In [11]:
# Calculate PSNR between input.mp4 and output.mp4
!ffmpeg -hide_banner -i output.mp4 -i input.mp4 -lavfi psnr -f null - 2>&1 | grep "Parsed_psnr_0"

# Calculate SSIM between input.mp4 and output.mp4
!ffmpeg -hide_banner -i output.mp4 -i input.mp4 -lavfi ssim -f null - 2>&1 | grep "Parsed_ssim_0"

# Calculate VMAF between input.mp4 (ref) and output.mp4 (target).
!ffmpeg -hide_banner -i output.mp4 -i input.mp4 \
  -lavfi "libvmaf=model_path=./model/vmaf_v0.6.1.json" -f null - 2>&1 | grep 'VMAF' \
  || echo 'VMAF calculation failed!'

print('Done!')

[Parsed_psnr_0 @ 0x6fa5640] PSNR y:35.196985 u:41.514128 v:43.315852 average:36.556118 min:33.705087 max:41.320742
[Parsed_ssim_0 @ 0x7311640] SSIM Y:0.949493 (12.966453) U:0.952224 (13.207927) V:0.967040 (14.820154) All:0.952873 (13.267257)
[libvmaf @ 0x5fcdc80] VMAF score: 85.662419
Done!


In [14]:
!ffmpeg -h

ffmpeg version 5.0.1-static https://johnvansickle.com/ffmpeg/  Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 8 (Debian 8.3.0-6)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-debug --disable-ffplay --disable-indev=sndio --disable-outdev=sndio --cc=gcc --enable-fontconfig --enable-frei0r --enable-gnutls --enable-gmp --enable-libgme --enable-gray --enable-libaom --enable-libfribidi --enable-libass --enable-libvmaf --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-librubberband --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libvorbis --enable-libopus --enable-libtheora --enable-libvidstab --enable-libvo-amrwbenc --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libdav1d --enable-libxvid --enable-libzvbi --enable-libzimg
  libavutil      57. 17.100 / 57. 17.100
  libavcodec     59. 18.100 / 59. 18.100
  libavformat   